In [7]:
#import pandas as pd
import dask.dataframe as dd
import dask.array as da
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('sentencizer')
from tqdm.notebook import tqdm 
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
from adjustText import adjust_text
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from gensim.models.phrases.Phraser(bigram)

In [6]:
df = dd.read_csv('TBG_unique_raw.csv', 
                 usecols = ['position_section','hl1', 'hl2', 'lede', 'body'], 
                 dtype = {'position_section': str,'hl1': object, 
                          'hl2': object, 'lede': str, 'body': str})
df = df.map_partitions(lambda x: x.fillna(''))
df['content'] = df.map_partitions(lambda x: x[['hl1', 'hl2', 'lede', 'body']].agg(' '.join, axis=1))
df = df.drop(columns = ['hl1', 'hl2', 'lede', 'body'])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/benbadnani/Desktop/Spring 2022/BU Spark/TBG_unique_raw.csv'

In [103]:
def custom_preprocess(corpus_iterable):
    global id2word
    pre_processed_corpus = []
    temp = []
    for x in tqdm(nlp.pipe(corpus_iterable), total = len(corpus_iterable)):
        for y in x.sents: 
            temp.append('<s>')
            for z in y: 
                if z.is_word and not z.is_stop:
                    temp.append(z.lemma_.lower())
            temp.append('</s>')
        pre_processed_corpus.append(temp)
        temp = []
        
    id2word.add_documents(pre_processed_corpus)

In [15]:
id2word = Dictionary()

In [104]:
arr = df.content.values
with nlp.select_pipes(disable=['tok2vec', 'parser', 'ner']):
    pre_processed_corpus = arr.map_blocks(lambda x: custom_preprocess(x), 
                                          new_axis = 0, dtype = object)

0it [00:00, ?it/s]

In [ ]:
pre_processed_corpus.compute()

In [ ]:
id2word.filter_extremes(no_below=15, no_above=0.4, keep_n=80000)

In [ ]:
lda_model = LdaModel(id2word=id2word, 
                     num_topics=7, alpha='auto', eval_every=5, per_word_topics=True, chunksize = 100, passes=10)

In [99]:
pre_processed_corpus

dask.array<lambda, shape=(1, nan), dtype=object, chunksize=(1, nan), chunktype=numpy.ndarray>

In [11]:
pca = PCA(n_components=2)

def plot_figure(reduced_matrix, name): 
    fig, ax = plt.subplots()
    plt.scatter(reduced_matrix[:,0], reduced_matrix[:,1])
    texts =\
    [plt.text(reduced_matrix[i, 0], reduced_matrix[i, 1], 
              word, ha='center', va='center') for i, word in enumerate(plays)]
    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='red'))
    plt.savefig(f'{name}.pdf')
    plt.show()